In [1]:
// Set log level to ERROR (less verbose)
sc.setLogLevel("ERROR")

Intitializing Scala interpreter ...

Spark Web UI available at http://252f7d7c2f69:4040
SparkContext available as 'sc' (version = 2.4.2, master = local[*], app id = local-1559548881171)
SparkSession available as 'spark'


In [2]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [3]:
val users: RDD[(VertexId, (String, String))] = sc.parallelize(Array(
    (3L, ("rxin", "student")), 
    (7L, ("jgonzal", "postdoc")), 
    (5L, ("franklin", "prof")), 
    (2L, ("istoica", "prof"))))

users.foreach(println)

(2,(istoica,prof))
(3,(rxin,student))
(5,(franklin,prof))
(7,(jgonzal,postdoc))


users: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at <console>:29


In [4]:
val relationships: RDD[Edge[String]] = sc.parallelize(Array(
    Edge(3L, 7L, "collab"), 
    Edge(5L, 3L, "advisor"), 
    Edge(2L, 5L, "colleague"), 
    Edge(5L, 7L, "pi")))

relationships.foreach(println)

Edge(3,7,collab)
Edge(2,5,colleague)
Edge(5,7,pi)
Edge(5,3,advisor)


relationships: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at parallelize at <console>:29


In [5]:
val defaultUser = ("John Doe", "Missing")

defaultUser: (String, String) = (John Doe,Missing)


In [6]:
val graph = Graph(users, relationships, defaultUser)

graph: org.apache.spark.graphx.Graph[(String, String),String] = org.apache.spark.graphx.impl.GraphImpl@4f942e9d


In [7]:
graph.vertices.foreach(println)

(5,(franklin,prof))
(2,(istoica,prof))
(3,(rxin,student))
(7,(jgonzal,postdoc))


In [8]:
graph.edges.foreach(println)

Edge(3,7,collab)
Edge(5,3,advisor)
Edge(2,5,colleague)
Edge(5,7,pi)


In [9]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import scala.util.hashing.MurmurHash3
import org.apache.spark.SparkContext._

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import scala.util.hashing.MurmurHash3
import org.apache.spark.SparkContext._


! wget http://stat-computing.org/dataexpo/2009/2008.csv.bz2

In [10]:
val df = spark
  .read
  .option("header","true")
  .csv("2008.csv.bz2")

df: org.apache.spark.sql.DataFrame = [Year: string, Month: string ... 27 more fields]


In [17]:
df.columns

res11: Array[String] = Array(Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, ArrTime, CRSArrTime, UniqueCarrier, FlightNum, TailNum, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, DepDelay, Origin, Dest, Distance, TaxiIn, TaxiOut, Cancelled, CancellationCode, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay)


In [16]:
df.select(
"FlightNum",
"Month",
"DayofMonth",
"DepTime",
"ArrTime",
"Origin",
"Dest").show()

+---------+-----+----------+-------+-------+------+----+
|FlightNum|Month|DayofMonth|DepTime|ArrTime|Origin|Dest|
+---------+-----+----------+-------+-------+------+----+
|      335|    1|         3|   2003|   2211|   IAD| TPA|
|     3231|    1|         3|    754|   1002|   IAD| TPA|
|      448|    1|         3|    628|    804|   IND| BWI|
|     1746|    1|         3|    926|   1054|   IND| BWI|
|     3920|    1|         3|   1829|   1959|   IND| BWI|
|      378|    1|         3|   1940|   2121|   IND| JAX|
|      509|    1|         3|   1937|   2037|   IND| LAS|
|      535|    1|         3|   1039|   1132|   IND| LAS|
|       11|    1|         3|    617|    652|   IND| MCI|
|      810|    1|         3|   1620|   1639|   IND| MCI|
|      100|    1|         3|    706|    916|   IND| MCO|
|     1333|    1|         3|   1644|   1845|   IND| MCO|
|      829|    1|         3|   1426|   1426|   IND| MDW|
|     1016|    1|         3|    715|    720|   IND| MDW|
|     1827|    1|         3|   

In [19]:
val airportCodes = df.select($"Origin", $"Dest")
  .flatMap(x => Iterable(x(0).toString, x(1).toString))

airportCodes.show()

+-----+
|value|
+-----+
|  IAD|
|  TPA|
|  IAD|
|  TPA|
|  IND|
|  BWI|
|  IND|
|  BWI|
|  IND|
|  BWI|
|  IND|
|  JAX|
|  IND|
|  LAS|
|  IND|
|  LAS|
|  IND|
|  MCI|
|  IND|
|  MCI|
+-----+
only showing top 20 rows



airportCodes: org.apache.spark.sql.Dataset[String] = [value: string]


In [23]:
val airportVertices: RDD[(VertexId, String)] = airportCodes
  .rdd
  .distinct()
  .map(x => (MurmurHash3.stringHash(x), x))

airportVertices.toDF.show()

+-----------+---+
|         _1| _2|
+-----------+---+
|-1128481959|HHH|
|-1717145497|SYR|
|-2105158920|ATW|
| 1342016723|LBB|
|  682905919|OGD|
|-1841167711|PIA|
| 1707640005|ABY|
|  891784594|SUN|
|-2029236616|TLH|
|-1852287689|RIC|
|-1405933338|EWN|
| 1014929201|CLE|
|  256932131|GFK|
|  868865441|BOI|
| 1294756036|BIS|
|-1162389404|RFD|
| 1291847767|FSM|
| 2103479318|DLH|
| 1917476200|EWR|
| 1169375524|IAD|
+-----------+---+
only showing top 20 rows



airportVertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String)] = MapPartitionsRDD[52] at map at <console>:41


In [24]:
val defaultAirport = ("Missing")

defaultAirport: String = Missing


In [26]:
val flightsFromTo = df.select($"Origin",$"Dest")

flightsFromTo.show()

+------+----+
|Origin|Dest|
+------+----+
|   IAD| TPA|
|   IAD| TPA|
|   IND| BWI|
|   IND| BWI|
|   IND| BWI|
|   IND| JAX|
|   IND| LAS|
|   IND| LAS|
|   IND| MCI|
|   IND| MCI|
|   IND| MCO|
|   IND| MCO|
|   IND| MDW|
|   IND| MDW|
|   IND| MDW|
|   IND| MDW|
|   IND| PHX|
|   IND| PHX|
|   IND| TPA|
|   ISP| BWI|
+------+----+
only showing top 20 rows



flightsFromTo: org.apache.spark.sql.DataFrame = [Origin: string, Dest: string]


In [29]:
val flightEdges = flightsFromTo.map(x => ((
    MurmurHash3.stringHash(x(0).toString),
    MurmurHash3.stringHash(x(1).toString)),
    1))
    .rdd
    .reduceByKey(_+_)
    .map(x => Edge(x._1._1, x._1._2, x._2))

flightEdges.toDF.show()

+-----------+-----------+-----+
|      srcId|      dstId| attr|
+-----------+-----------+-----+
|-1331760633|-1020692585|  653|
| -297351069|  580398961|  124|
| 1917476200|  813811672| 3729|
|-1352361520|-1227538930|  549|
|-1011165063| -499363706| 1401|
|  -47892603| 1014929201|    2|
| -520037749|-1963423298| 1525|
|-1020692585| 1058030826|   94|
|-1093474298| 1364248589|   15|
| 1226329766| 1146947515|    2|
|-1515187592| 1710100676| 3575|
| 1226329766| 1554094361|  338|
|  -44023576|-1011165063|  366|
| -520037749| 1364248589|10770|
|  178004914| 1838031709|  155|
|  -44023576|-1954981156| 1144|
|-1182139023| 1838031709| 1474|
|-1345915352|  285116926|  741|
|  813811672|-1276676011| 1777|
| 1710100676| 1800873072|  119|
+-----------+-----------+-----+
only showing top 20 rows



flightEdges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[74] at map at <console>:46


In [30]:
val graph = Graph(airportVertices, flightEdges, defaultAirport)

graph: org.apache.spark.graphx.Graph[String,Int] = org.apache.spark.graphx.impl.GraphImpl@35ddad6c


In [32]:
graph.edges.take(10).foreach(println)

Edge(-2119762115,-1954981156,285)
Edge(-2119762115,-1895512637,1036)
Edge(-2119762115,-1745975158,60)
Edge(-2119762115,-1736145510,1418)
Edge(-2119762115,-1652015379,306)
Edge(-2119762115,-57532706,1)
Edge(-2119762115,-6572541,23)
Edge(-2119762115,265243917,2498)
Edge(-2105183870,-227228204,173)
Edge(-2105183870,813811672,60)


In [43]:
graph.vertices.filter{ case (vid, airportCodes) => airportCodes == "LAX" }.take(10) //-44023576

res32: Array[(org.apache.spark.graphx.VertexId, String)] = Array((-44023576,LAX))


In [53]:
graph.edges.filter{e => e.srcId == -44023576 }.take(10).foreach(println)

Edge(-44023576,-1963423298,994)
Edge(-44023576,-1954981156,1144)
Edge(-44023576,-1885570066,548)
Edge(-44023576,-1753379389,177)
Edge(-44023576,-1339998817,1291)
Edge(-44023576,-1082293340,8058)
Edge(-44023576,-1011165063,366)
Edge(-44023576,-216730884,1038)
Edge(-44023576,-170780705,373)
Edge(-44023576,-57532706,11257)


In [54]:
graph.edges.filter{e => e.srcId == -44023576 }.count

res41: Long = 90


In [62]:
graph.triplets.map(triplet => triplet.attr
  + "\tflights from "
  + triplet.srcAttr 
  + " to " 
  + triplet.dstAttr).collect.foreach(println(_))

285	flights from SNA to CVG
1036	flights from SNA to MSP
60	flights from SNA to RNO
1418	flights from SNA to PDX
306	flights from SNA to STL
1	flights from SNA to SAN
23	flights from SNA to LIH
2498	flights from SNA to SMF
173	flights from BMI to MCO
60	flights from BMI to DFW
90	flights from ATW to MSP
1	flights from ATW to DSM
2288	flights from ATW to ORD
219	flights from TLH to MCO
831	flights from TLH to MIA
2	flights from PIR to JAX
366	flights from AMA to ABQ
2669	flights from AMA to DAL
809	flights from AMA to IAH
364	flights from BRW to FAI
1519	flights from COS to ORD
1	flights from COS to CLE
1	flights from COS to FAT
117	flights from COS to MEM
1	flights from COS to LAN
591	flights from COS to ATL
23	flights from CVG to SCE
491	flights from CVG to IND
852	flights from CVG to BHM
345	flights from CVG to STL
160	flights from CVG to BUF
653	flights from CVG to AUS
1235	flights from CVG to BWI
795	flights from CVG to CMH
1671	flights from CVG to PHL
1532	flights from CVG to JFK


3402	flights from TPA to BWI
1692	flights from TPA to DEN
204	flights from TPA to DAY
235	flights from TPA to SWF
3051	flights from TPA to FLL
136	flights from TPA to PNS
1452	flights from TPA to DTW
741	flights from TPA to MHT
1067	flights from TPA to RDU
1353	flights from TPA to PBI
860	flights from TPA to MEM
488	flights from TPA to SDF
1906	flights from TPA to IAD
1093	flights from TPA to ISP
366	flights from TPA to SJU
699	flights from TPA to MCI
646	flights from IPL to LAX
653	flights from HSV to DEN
1043	flights from HSV to IAH
248	flights from HSV to MCO
1	flights from CLT to ATW
877	flights from CLT to CVG
136	flights from CLT to PDX
747	flights from CLT to SYR
243	flights from CLT to MKE
2988	flights from CLT to DCA
2349	flights from CLT to PIT
661	flights from CLT to SAT
3295	flights from CLT to FLL
1144	flights from CLT to MDT
2340	flights from CLT to RDU
1971	flights from CLT to CLE
5339	flights from CLT to LGA
1183	flights from CLT to MCI
1420	flights from MOB to IAH
1094

1212	flights from MHT to CLE
732	flights from HNL to LAS
366	flights from HNL to DEN
702	flights from HNL to IAH
732	flights from HNL to DFW
372	flights from EUG to PHX
1	flights from EUG to RDM
1927	flights from FSD to MSP
1	flights from FSD to MSN
1	flights from FSD to SDF
334	flights from ASE to LAX
1342	flights from EVV to DTW
2853	flights from FAI to ANC
793	flights from CAK to LGA
2958	flights from FWA to ORD
1	flights from FWA to TVC
1202	flights from FWA to DTW
714	flights from LRD to IAH
1993	flights from ACT to DFW
365	flights from SBN to MSP
435	flights from MRY to LGB
246	flights from MRY to ONT
2290	flights from ELP to DAL
670	flights from ELP to ORD
4136	flights from ELP to PHX
366	flights from ELP to SAN
381	flights from ELP to ONT
1033	flights from ELP to HOU
2	flights from PUB to COS
30	flights from EGE to CVG
13	flights from EGE to JFK
197	flights from EGE to ORD
249	flights from TRI to CVG
16	flights from TRI to CLT
1425	flights from ABE to ORD
643	flights from GRK t

3045	flights from VPS to ATL
273	flights from ONT to RNO
248	flights from ONT to JFK
440	flights from ONT to GEG
1599	flights from ONT to SFO
365	flights from ONT to BNA
228	flights from ONT to TUL
246	flights from ONT to MRY
1	flights from ONT to GJT
471	flights from ONT to BOI
238	flights from ONT to FAT
302	flights from ONT to MCI
194	flights from TEX to PHX
1932	flights from HOU to CRP
659	flights from HOU to PHL
1722	flights from HOU to PHX
1416	flights from HOU to LAX
1	flights from HOU to BTR
3595	flights from HOU to ATL
1	flights from BTV to IND
346	flights from BTV to MCO
826	flights from BTV to DTW
614	flights from BTV to CLE
129	flights from BTV to IAD
500	flights from BTV to ATL
1	flights from LWS to IDA
5368	flights from ATL to MSP
9	flights from ATL to TUP
1114	flights from ATL to DSM
5	flights from ATL to CRP
4193	flights from ATL to RSW
9	flights from ATL to MFE
674	flights from ATL to MEI
3610	flights from ATL to MKE
5978	flights from ATL to BWI
7311	flights from ATL t

226	flights from OKC to SMF
21	flights from OKC to CLE
5	flights from OKC to IAD
1353	flights from OKC to HOU
35	flights from EYW to MCO
361	flights from PHF to MCO
716	flights from PHF to BOS
4157	flights from PHF to ATL
2	flights from STX to JAX
3955	flights from SFO to SNA
364	flights from SFO to COS
932	flights from SFO to OGG
1636	flights from SFO to PSP
609	flights from SFO to AUS
2	flights from SFO to SMX
529	flights from SFO to ABQ
2570	flights from SFO to MFR
5374	flights from SFO to DEN
2660	flights from SFO to IAH
4870	flights from SFO to PHX
6312	flights from SFO to SAN
483	flights from SFO to LIH
1923	flights from SFO to EUG
354	flights from SFO to OTH
1082	flights from SFO to RDM
4064	flights from SFO to SLC
2337	flights from SFO to ACV
3098	flights from SFO to ATL
12	flights from SFO to FCA
3137	flights from SFO to EWR
586	flights from AVP to ATL
731	flights from DEN to AMA
1384	flights from DEN to CVG
5011	flights from DEN to MSP
2473	flights from DEN to DRO
1631	flight

...

3626	flights from HPN to ORD
2158	flights from HPN to ATL
766	flights from CHA to ORD
426	flights from CHA to DFW
1826	flights from SNA to IAH
33	flights from SNA to LAX
2015	flights from SNA to SLC
954	flights from SNA to EWR
1871	flights from FLG to PHX
50	flights from BMI to LAS
743	flights from ATW to ATL
1	flights from LMT to MFR
3	flights from PIR to MSP
450	flights from COS to MSP
1	flights from COS to MKE
1	flights from COS to ABQ
364	flights from COS to SFO
460	flights from COS to ONT
104	flights from CVG to COS
1745	flights from CVG to MSP
786	flights from CVG to RIC
921	flights from CVG to LAS
444	flights from CVG to SRQ
800	flights from CVG to HSV
777	flights from CVG to SFO
357	flights from CVG to AVP
882	flights from CVG to SAT
571	flights from CVG to SAN
450	flights from CVG to ORF
101	flights from CVG to XNA
3023	flights from CVG to DTW
776	flights from CVG to MHT
135	flights from CVG to EVV
140	flights from CVG to ILM
247	flights from CVG to TRI
127	flights from CVG to

366	flights from MAF to LAS
471	flights from MAF to AUS
1187	flights from MAF to IAH
676	flights from MAF to HOU
2380	flights from AGS to ATL
366	flights from ABQ to STL
433	flights from ABQ to AUS
366	flights from ABQ to TPA
367	flights from ABQ to MAF
3078	flights from ABQ to DAL
229	flights from ABQ to OKC
424	flights from ABQ to SAT
2261	flights from ABQ to IAH
2395	flights from ABQ to LAX
1016	flights from ABQ to OAK
466	flights from ABQ to SMF
2888	flights from ABQ to DFW
12	flights from ABQ to CLE
1067	flights from ABQ to ATL
670	flights from ABQ to MCI
1316	flights from SJT to DFW
1583	flights from PHL to CVG
853	flights from PHL to IND
578	flights from PHL to STL
774	flights from PHL to MKE
290	flights from PHL to AUS
660	flights from PHL to BWI
2237	flights from PHL to SFO
981	flights from PHL to BNA
448	flights from PHL to ALB
1732	flights from PHL to IAH
2977	flights from PHL to FLL
6384	flights from PHL to BOS
817	flights from PHL to SAN
2652	flights from PHL to MDW
2357	f

144	flights from JAX to CLE
366	flights from JAX to HOU
5656	flights from JAX to ATL
793	flights from KOA to OGG
713	flights from KOA to SFO
411	flights from KOA to SEA
4	flights from KOA to OAK
9038	flights from KOA to HNL
1392	flights from PHX to COS
4754	flights from PHX to BUR
367	flights from PHX to BHM
2183	flights from PHX to CLT
8402	flights from PHX to DEN
5130	flights from PHX to ORD
1123	flights from PHX to BNA
7609	flights from PHX to SAN
9992	flights from PHX to LAX
2929	flights from PHX to DTW
1962	flights from PHX to YUM
692	flights from PHX to ICT
365	flights from PHX to MHT
914	flights from PHX to HNL
1037	flights from PHX to GJT
334	flights from PHX to BDL
1618	flights from PHX to SBA
365	flights from PHX to SDF
7177	flights from PHX to SLC
3684	flights from PHX to TUS
1677	flights from PHX to HOU
2140	flights from PHX to MCI
2060	flights from PHX to OMA
578	flights from FLL to IND
366	flights from FLL to RIC
328	flights from FLL to SYR
43	flights from FLL to MKE
593	

1157	flights from DFW to CAE
3486	flights from DFW to SFO
1247	flights from DFW to TXK
2861	flights from DFW to BNA
5414	flights from DFW to SAT
3915	flights from DFW to SEA
1897	flights from DFW to CID
3036	flights from DFW to BOS
1035	flights from DFW to ORF
6571	flights from DFW to LAX
109	flights from DFW to GUC
971	flights from DFW to GGG
732	flights from DFW to HNL
1994	flights from DFW to ACT
1154	flights from DFW to CHS
2347	flights from DFW to RDU
1665	flights from DFW to CLE
2501	flights from DFW to MEM
2010	flights from DFW to IAD
176	flights from DFW to ROC
1435	flights from DFW to GSP
1747	flights from DFW to OMA
215	flights from GPT to ORD
1908	flights from GPT to IAH
3441	flights from GPT to ATL
349	flights from MIA to STX
1097	flights from MIA to SFO
2734	flights from MIA to IAH
366	flights from MIA to PHX
2193	flights from MIA to BOS
154	flights from MIA to XNA
43	flights from MIA to EGE
1098	flights from MIA to RDU
399	flights from MIA to CLE
3587	flights from MIA to 

1522	flights from FNT to DTW
365	flights from HLN to MSP
412	flights from HLN to DEN
26	flights from SJU to MSP
245	flights from SJU to STT
3558	flights from SJU to JFK
1	flights from SJU to PIT
730	flights from SJU to IAH
2415	flights from SJU to MCO
43	flights from SJU to CLE
12	flights from FCA to SFO
668	flights from FCA to DEN
90	flights from ANC to CVG
27	flights from ANC to OGG
826	flights from ANC to MSP
363	flights from ANC to SCC
207	flights from ANC to IAH
102	flights from ANC to ADK
6257	flights from ANC to SEA
3217	flights from ANC to FAI
115	flights from ANC to ATL
232	flights from EWR to STT
681	flights from EWR to BHM
1873	flights from EWR to RSW
2350	flights from EWR to LAS
508	flights from EWR to BWI
5945	flights from EWR to CLT
1653	flights from EWR to CMH
15	flights from EWR to HDN
1092	flights from EWR to GSO
949	flights from EWR to SAT
3989	flights from EWR to IAH
2601	flights from EWR to PHX
4432	flights from EWR to FLL
658	flights from EWR to AVL
2103	flights fr